In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [60]:
class Pretrain(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [61]:
class FF(nn.Module):
    def __init__(self, model):
        super().__init__()
        
        self.cnn = model
        
        self.fc = nn.Linear(20, 1)
        
    def forward(self, x):
        x1 = x[:,:1,:]
        x2 = x[:,1:2,:]
        

        x1 = self.cnn(x1)
        x2 = self.cnn(x2)
            
        x = self.fc(torch.cat((x1, x2), dim=1))
        
        return x

In [62]:
model = Pretrain()

In [63]:
from dataloading import load_data

In [64]:
trainloader, testloader = load_data()

In [65]:
learning_rate = .001

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(baseline.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [66]:
epochs = 10

In [67]:
losses = []

for epoch in range(epochs):
    running_loss = 0.0
    epoch_losses = []
    
    for i, data in enumerate(trainloader, 0):
        inputs, target, classes = data
        
        x1 = inputs[:,:1,:]
        x2 = inputs[:,1:2,:]
        
        optimizer.zero_grad()
        
        outputs = model(x1)
        outputs = outputs
        
        loss = criterion(outputs, classes[:,0])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        epoch_losses.append(loss.item())
    
    epoch_loss = torch.mean(torch.tensor(epoch_losses))
    print(f'epoch: {epoch}, loss: {epoch_loss}')
    losses.append(epoch_loss)  

epoch: 0, loss: 4.313869953155518
epoch: 1, loss: 0.6634032130241394
epoch: 2, loss: 0.3630056083202362
epoch: 3, loss: 0.2072702944278717
epoch: 4, loss: 0.14497286081314087
epoch: 5, loss: 0.09998053312301636
epoch: 6, loss: 0.058640483766794205
epoch: 7, loss: 0.03776092454791069
epoch: 8, loss: 0.018457647413015366
epoch: 9, loss: 0.011411981657147408


In [68]:
ff = FF(model)

In [69]:
learning_rate = .001

criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(baseline.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(ff.parameters(), lr=learning_rate)

In [70]:
losses = []

for epoch in range(epochs):
    running_loss = 0.0
    epoch_losses = []
    
    for i, data in enumerate(trainloader, 0):
        inputs, target, classes = data
        
        optimizer.zero_grad()
        
        outputs = ff(inputs)
        outputs = outputs.squeeze()
        
        loss = criterion(outputs, target.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        epoch_losses.append(loss.item())
    
    epoch_loss = torch.mean(torch.tensor(epoch_losses))
    print(f'epoch: {epoch}, loss: {epoch_loss}')
    losses.append(epoch_loss)  

epoch: 0, loss: 0.8037532567977905
epoch: 1, loss: 0.6821638941764832
epoch: 2, loss: 0.5796943306922913
epoch: 3, loss: 0.5355087518692017
epoch: 4, loss: 0.49673691391944885
epoch: 5, loss: 0.4737986922264099
epoch: 6, loss: 0.4638371169567108
epoch: 7, loss: 0.4517482817173004
epoch: 8, loss: 0.44714680314064026
epoch: 9, loss: 0.43703028559684753


In [72]:
from evaluate import evaluate_model

evaluate_model(ff, testloader)

Accuracy: 0.7950000166893005
